In [ ]:
import sys
!{sys.executable} -m pip install pymoo  # For ipopt solver

# Install a pip package in the current Jupyter kernel

from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

restartkernel()

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os
from spyci import spyci
import matplotlib.pyplot as plt
%matplotlib inline

# Write a file 2nd_order_rc_test.spice

In [56]:
%%writefile 2nd_order_rc_test.spice
***** 2nd Order RC Circuit Frequency Response *****
.PARAM R_1 = 2096.530184823837
.PARAM C_1 = 5.710242851887112e-08
.PARAM START = 1
.PARAM STOP = 1000000000.0
**--**--**--**--**--**--**--**--**--**
r1 1 2 {R_1}
c1 2 0 {C_1}
r2 2 3 {R_2}
c2 3 0 {C_2}


********Specifying an AC source with zero dc
vin 1 0 dc 0 ac 1
********AC analysis for 1 Hz to 1MHz, 10 points per decade
.ac dec 10 {START} {STOP}


.control
set filetype=ascii
run

let m=vdb(3)

*plot m xlog

meas ac cutoff find frequency WHEN m=-3.0 CROSS=LAST

write 2nd_order_rc_test.raw cutoff
.endc
.end

Writing 2nd_order_rc_test.spice


In [65]:
filename="2nd_order_rc_test.raw"
data = spyci.load_raw(filename)  # see 'Data structure' section below
spyci.list_vars(filename)

Variables:

  idx  name    type
-----  ------  -------------
    1  cutoff  notype dims=1


In [58]:
def add_param_lines(file_path, param_names_and_values):
    
  if not os.path.exists(file_path):
    raise FileNotFoundError("File does not exist: {}".format(file_path))

  with open(file_path, "r") as f:
    lines = f.readlines()

  first_line = lines[0]

  param_lines = []
  for param_name, param_value in param_names_and_values:
    param_lines.append(".PARAM {} = {}\n".format(param_name, param_value))

  # Find the index of the line with the text `"**--**--**--**--**--**--**--**--**--**\n`
  recognizer="**--**--**--**--**--**--**--**--**--**\n"
  try:
      index = lines.index(recognizer)
      new_lines = [first_line] + param_lines + [recognizer] + lines[index + 1:]
  except ValueError:
      new_lines = [first_line] + param_lines + [recognizer] + lines[1:]

  with open(file_path, "w") as f:
    f.writelines(new_lines)


In [59]:
def get_values(filename, pvars=None):
    """
    Load the values from a SPICE raw file and return a list of arrays.

    Args:
        filename (str): The path to the SPICE raw file.
        pvars (list): A list of string variables to load. If None, all variables are loaded.

    Returns:
        list: A list of arrays, one for each variable in pvars.
    """

    d = spyci.load_raw(filename)
    xx = []
    for var in pvars:
        xx.append(d['values'][var].real)
    return xx

# Test the add_param_function

In [60]:
filename = "2nd_order_rc_test.spice"
param_names_and_values = [
    
    ["R_1",   1e3], 
    ["C_1",   1e-6],
    ["R_2",   1e3], 
    ["C_2",   1e-6],
    ["START", 1], 
    ["STOP",  1e8]
    
    ]
add_param_lines(filename, param_names_and_values)

# View the contents

In [61]:
r=os.system("cat 2nd_order_rc_test.spice")

***** 2nd Order RC Circuit Frequency Response *****
.PARAM R_1 = 1000.0
.PARAM C_1 = 1e-06
.PARAM R_2 = 1000.0
.PARAM C_2 = 1e-06
.PARAM START = 1
.PARAM STOP = 100000000.0
**--**--**--**--**--**--**--**--**--**
r1 1 2 {R_1}
c1 2 0 {C_1}
r2 2 3 {R_2}
c2 3 0 {C_2}


********Specifying an AC source with zero dc
vin 1 0 dc 0 ac 1
********AC analysis for 1 Hz to 1MHz, 10 points per decade
.ac dec 10 {START} {STOP}


.control
set filetype=ascii
run

let m=vdb(3)

*plot m xlog

meas ac cutoff find frequency WHEN m=-3.0 CROSS=LAST

write 2nd_order_rc_test.raw cutoff
.endc
.end


# Test the get_values function

In [64]:
get_values("2nd_order_rc_test.raw",['cutoff'])


[array([116.9059,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    ,   0.    ,   0.    ,
          0.    ,   0.    ,   0.    ,   0.    , 

In [66]:
import os
def run_spice_again(x,filename):
    param_names_and_values = [
    
        ["R_1",   x[0]], 
        ["C_1",   x[1]],
        ["R_2",   x[2]], 
        ["C_2",   x[3]],
        ["START", 1], 
        ["STOP",  1e9]
        ]
    add_param_lines(filename, param_names_and_values)
    
    os.system("ngspice -b 2nd_order_rc_test.spice -o .temp > .temp")
    os.system("rm -f .temp")
    filename="2nd_order_rc_test.raw"
    data = spyci.load_raw(filename)  # see 'Data structure' section below
    cutoff_value=get_values(filename,['cutoff'])[0][0]
#     print(cutoff_value)
    return cutoff_value

print(run_spice_again([1.2e3,1e-6,1.8e3,1e-7],"2nd_order_rc_test.spice"))

116.9059


In [67]:
import numpy as np
from pymoo.problems.functional import FunctionalProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.termination import get_termination
from pymoo.optimize import minimize

def find_values(n_var,x,range_l,range_u,Target_F_cut,objs,max_generations,verbose=True):

  # now put everything together to a problem object
  problem = FunctionalProblem(n_var,
                              objs,
                              # constr_ieq=constr_ieq,
                              xl=np.array(range_l),
                              xu=np.array(range_u)
                              )
  # F= problem.evaluate([x])
  # print(f"F: {F}\n")
  ############
  algorithm = NSGA2(
    pop_size=45,
    n_offsprings=10,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=15),
    eliminate_duplicates=True
  )
  ##############
  termination = get_termination("n_gen", max_generations)
  ##############
  res = minimize(problem,
               algorithm,
               termination,
               seed=9,
               save_history=True,
               verbose=verbose)
  
  return res


In [68]:
#########################
range_l=[1000, 1e-9, 1000, 1e-9]
range_u=[4000, 100e-9,5000, 100e-9]
target_f=370
current_x=[1800,88.2097e-9,1800,88.2097e-9]## correct for 1000Hz
max_generations=40
verbose=True

run_spice_again(current_x,"2nd_order_rc_test.spice")
#########################COST_FUNCTION##########################################
cost_function = [
    lambda x: (run_spice_again(x,"2nd_order_rc_test.spice")-target_f)**2 ########  x[0]=R,x[1]=C
]
################################################################################

r=find_values(4,current_x,range_l,range_u, target_f,cost_function,max_generations,verbose=verbose)

result=r.X.T
result


n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       45 |      1 |             - |             -
     2 |       55 |      1 |  8.6797758000 |         ideal
     3 |       65 |      1 |  0.000000E+00 |             f
     4 |       75 |      1 |  0.000000E+00 |             f
     5 |       85 |      1 |  0.000000E+00 |             f
     6 |       95 |      1 |  0.000000E+00 |             f
     7 |      105 |      1 |  1.265934E+01 |         ideal
     8 |      115 |      1 |  0.000000E+00 |             f
     9 |      125 |      1 |  0.000000E+00 |             f
    10 |      135 |      1 |  0.000000E+00 |             f
    11 |      145 |      1 |  0.000000E+00 |             f
    12 |      155 |      1 |  0.000000E+00 |             f
    13 |      165 |      1 |  0.000000E+00 |             f
    14 |      175 |      1 |  0.000000E+00 |             f
    15 |      185 |      1 |  0.000000E+00 |             f
    16 |      195 |      1 |  0.000000E+00 |            

array([1.73806424e+03, 5.98730551e-09, 3.70981699e+03, 7.81481054e-08])

In [74]:
R_opt1=result[0]  #1kOhm
C_opt1=result[1]  # 1µF
R_opt2=result[2]  #1kOhm
C_opt2=result[3]  # 1µF
# F_cut = 1./(2.*np.pi*R_opt*C_opt)
print("========================================")
print("Target cut off frequency is: {:7.2f} Hz".format(target_f))
# print("Obtained cut off frequency(Optimized value ) is: {:7.2f} Hz".format(F_cut))
# print("Percentage Error is: {:7.2f} %".format(((F_cut-target_f)*100/target_f)))
print("Resistance R1(Optimized value ) is: {:7.3f} Kilo Ohms".format(R_opt1/1e3))
print("Capacitance C1(Optimized value ) is: {:7.3f} nanofarads".format(C_opt1*1e9))
print("Resistance R2(Optimized value ) is: {:7.3f} Kilo Ohms".format(R_opt2/1e3))
print("Capacitance C2(Optimized value ) is: {:7.3f} nanofarads".format(C_opt2*1e9))
print("========================================")

Target cut off frequency is:  370.00 Hz
Resistance R1(Optimized value ) is:   1.738 Kilo Ohms
Capacitance C1(Optimized value ) is:   5.987 nanofarads
Resistance R2(Optimized value ) is:   3.710 Kilo Ohms
Capacitance C2(Optimized value ) is:  78.148 nanofarads
